## Setup

In [3]:
from dotenv import load_dotenv
from autogen import ConversableAgent

load_dotenv()
llm_config={"model": "gpt-4o-mini"}

In [4]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [6]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [7]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating tasks

In [8]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process

In [9]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/Users/danielboza/Documents/Projects/langgraph-experiments/.venv/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

Daniel from Maracaibo

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you, Daniel from Maracaibo! That’s all the information I need. 

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
```json
{'name': 'Daniel', 'location': 'Maracaibo'}
```

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Topic preference Agent):

Horses

------------------------

In [10]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

```json
{'name': 'Daniel', 'location': 'Maracaibo'}
```


Daniel, located in Maracaibo, is interested in reading about horses.


Daniel is interested in reading about horses. Fun facts shared include their ability to sleep standing up, and a joke about a horse changing its jockeys. A suggestion was made to read the legend of the “Headless Horseman” for an interesting story.




In [11]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 5.1149999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.1149999999999996e-05, 'prompt_tokens': 185, 'completion_tokens': 39, 'total_tokens': 224}}, 'usage_excluding_cached_inference': {'total_cost': 5.1149999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 5.1149999999999996e-05, 'prompt_tokens': 185, 'completion_tokens': 39, 'total_tokens': 224}}}


{'usage_including_cached_inference': {'total_cost': 1.9499999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 1.9499999999999996e-05, 'prompt_tokens': 70, 'completion_tokens': 15, 'total_tokens': 85}}, 'usage_excluding_cached_inference': {'total_cost': 1.9499999999999996e-05, 'gpt-4o-mini-2024-07-18': {'cost': 1.9499999999999996e-05, 'prompt_tokens': 70, 'completion_tokens': 15, 'total_tokens': 85}}}


{'usage_including_cached_inference': {'total_cost': 0.00020879999999999998, 'gpt-4o-mini-2024-07-18': {'cost': 0.00020879999999999998, 'prompt_tokens': 392, 'completion_tokens': 250,